# VAR Models

Notebook for testing Vector Autoregressive Models

## Problem Statement

Predict electricity prices in Spain for each hour of the upcoming day more accurately than estimates provided by the Spanish transmission agent and operator. 

Use information available during the 2pm-3pm window the previous day during which generators in Spain submit their bids. 

## Contents

- [Imports](#Imports)
- [Functions Used](#Functions-Used)
- [Prepare Data](#Prepare-Data)
- [VAR](#VAR-Model)
- [VARMAX](#VARMAX-Model)
- [Results for Analysis](#Results-for-Analysis)

### Imports

In [61]:
# General Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# General modeling imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics

# Models
from statsmodels.tsa.api import VAR, VARMAX

In [2]:
df = pd.read_csv('../Data/Analysis/model_data.csv')
visuals = pd.read_csv('../Data/intermediary/energy.csv')

In [3]:
# visuals contains the prices attached to their original hour
# and will be used to visualize and compare our predictions
visuals.set_index(pd.DatetimeIndex(visuals['time']), inplace=True)
visuals = visuals[['price_actual','price_day_ahead']]
visuals.head(2)

,price_actual,price_day_ahead
time,,
2015-01-01 00:00:00+00:00,65.41,50.1
2015-01-01 01:00:00+00:00,64.92,48.1


In [57]:
# Prediction df for appending test predictions too
pred_df = visuals[visuals.index.year == 2018]

### Functions Used

In [55]:
# Function for evaluating the regressions
# and outputting a dataframe of different metrics
# for each hour predicted
def reg_metrics(y_test, y_test_p, mod):
    test_rmse = np.sqrt(((y_test-y_test_p)**(2)).mean())
    test_r2 =  metrics.r2_score(y_test, y_test_p, multioutput='raw_values')
    metrics_df = pd.DataFrame(data = zip(test_rmse, test_r2),
                              columns=[mod+'test_rmse',mod+'test_r2'])
    return metrics_df

In [5]:
# function to convert predictions into dataframe for plotting
def append_preds(preds, previous_preds, name):
    new_preds = pd.DataFrame(np.ravel(preds),columns=[name], index=previous_preds.index)
    return previous_preds.join(new_preds)

### Prepare Data

In [6]:
# Set up data frame for modeling
# Drop time column
df.drop(columns=['time'], inplace=True)
# set index as date
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)
# sort index
df.sort_index(inplace=True)
# drop hour of day and date column
df.drop(columns=['hour_of_day','date'], inplace=True)

# Get columns for y
y_cols = [col for col in df.columns if col.startswith('t_price')]

# only need y for VAR model
y = df[y_cols]

# Train test split
train, test = train_test_split(y, shuffle=False)

### VAR Model

In [28]:
# Instantiate a VAR model
model = VAR(train, freq='D')

# Fit our model and use AIC
var_model = model.fit(maxlags=14, # at most a lag of 2 weeks
                     ic='aic')  # using AIC to determine the best model

# What is the order of our autoregressive model? 
var_model.k_ar

7

In [47]:
var_model.forecast(train.values,1)[0]

array([16.05319277, 11.11158413,  6.49468023,  5.47944946,  3.60792081,
        5.05233039, 10.46236915, 17.46071685, 23.25033608, 27.47544268,
       31.96112684, 34.13856574, 33.51396457, 32.67250808, 29.56730381,
       28.1215851 , 29.05994878, 34.14878639, 44.39632105, 51.17161541,
       53.00455629, 50.47241112, 43.26286055, 36.88436189])

Our basic VAR model chose lags of 7 (i.e. predicting off the same day the previous week). This is unsurprising given our EDA showing that prices varied by day of week, with the weekend having much lower prices than the the weekdays.

VAR models are generally good at predicting short term but begin to predict the mean when forecasting farther ahead in time. Thus we will build a process by which the VAR model continually predicts 1 value ahead and then retrains the model with the new data and predicts 1 value ahead. 

While this is not a perfect comparison with our other model types, it is likely that we will wish to have our final model to eventually engage in this sort of continuous learning and it will give us a better idea of how our model compares than trying to predict a year ahead using only the training data.

In [51]:
# Fit models and generate predictions
def continuous_var(train, test, lags = 7, criterium = 'aic'):
    # predictions array
    preds = []
    # fit first model and get predictions
    model = VAR(train, freq='D')
    var_model = model.fit(maxlags=lags, # at most a lag of 2 weeks
                          ic=criterium)
    preds.append(var_model.forecast(train.values, 1))
    # loop through test, appending to train & fitting + predicting
    for i in range(0, len(test)-1):
        train = train.append(test.loc[test.index[i],:])
        model = VAR(train, freq='D')
        var_model = model.fit(maxlags=lags, # at most a lag of 2 weeks
                         ic=criterium)
        preds.append(var_model.forecast(train.values, 1))
    return preds

In [52]:
# generate prediction
var_preds = continuous_var(train, test)

In [54]:
# predictions were nested too heavily, fix
var_preds_fixed = [var_preds[i][0] for i in range(len(var_preds))]

# convert to dataframe
var_preds_df = pd.DataFrame(var_preds_fixed, 
                            columns=test.columns,
                            index=test.index)
var_preds_df.head(2)

,t_price_0,t_price_1,t_price_2,t_price_3,t_price_4,t_price_5,t_price_6,t_price_7,t_price_8,t_price_9,...,t_price_14,t_price_15,t_price_16,t_price_17,t_price_18,t_price_19,t_price_20,t_price_21,t_price_22,t_price_23
date,,,,,,,,,,,,,,,,,,,,,
2017-12-31,16.053193,11.111584,6.494680,5.479449,3.607921,5.05233,10.462369,17.460717,23.250336,27.475443,...,29.567304,28.121585,29.059949,34.148786,44.396321,51.171615,53.004556,50.472411,43.262861,36.884362
2018-01-01,21.619322,15.886060,11.427035,9.757018,9.684822,13.09689,23.469110,34.782532,40.002809,40.554539,...,36.537260,35.589703,36.523356,43.063786,52.955654,54.370433,50.793200,49.463915,46.497647,38.696741


In [56]:
# Get metrics and view results
var_metrics = reg_metrics(test, var_preds_df, 'var_')
var_metrics.describe().T

,count,mean,std,min,25%,50%,75%,max
var_test_rmse,24.0,5.575370,0.929462,3.591614,5.098437,5.706196,6.413745,6.684455
var_test_r2,24.0,0.729843,0.109217,0.537102,0.695741,0.727999,0.819664,0.888508


This model is comprable to our SVR model as the best so far. Its mean test rmse and r-squared values are superior to the SVR model, however, the spread in those values is higher so it seems to be doing very well at predicting certain hours and more poorly at predicting others. We will investigate further in our combined model analyis notebook.

In [60]:
# Append predictions to pred df
pred_df = append_preds(var_preds_fixed, pred_df, 'var')
pred_df.head(2)

,price_actual,price_day_ahead,var
time,,,
2018-01-01 00:00:00+00:00,20.76,6.74,16.053193
2018-01-01 01:00:00+00:00,19.02,4.74,11.111584


### VARMAX Model

In [71]:
# Create exogenous variables
# since past prices are automatically part of a VARMAX model
# we will include oil price, and load forecast and wind forcast 
# as our exogenous variables

# get prediction columns
exog_cols = [col for col in df.columns if col.startswith('t_pred')]
exog_cols.append('oil_price')
# exogeneous df
exog = df[exog_cols]
# train test split
exog_train, exog_test = train_test_split(exog, shuffle=False)

In [ ]:
# Instantiate a VARMAX model, use 7 day lag, no moving average
model = VARMAX(endog=train, exog=exog_train, order=(7,0))

# Fit our model and use AIC
varmax_model = model.fit(ic='aic', maxiter=1000)  

# What is the order of our autoregressive model? 
varmx_model.k_ar

/Users/magnusbigelow/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
